# Task B

In [2]:
import pandas as pd
from string import punctuation

beerdf = pd.read_csv("6039 beer reviews of 250 beers.csv", sep="\t")
beerdf = beerdf.drop(columns = ["product_review","user_ratings","Unnamed: 0"],axis=1)
beerdf[:10]

,Product_name,overall_rating,clean_review
0,Kentucky Brunch Brand Stout,4.80,"['2016 Silver Wax. Aroma has whiskey, maple, t..."
1,Kentucky Brunch Brand Stout,4.74,['The beer pours Pitch Black with a frothy tan...
2,Kentucky Brunch Brand Stout,4.68,"[""Probably the smoothest beer I have ever had...."
3,Kentucky Brunch Brand Stout,5.00,"['Dark black, very thick, a little bit of tan ..."
4,Kentucky Brunch Brand Stout,4.97,['Poured black as ink with thin ruby edges at ...
5,Kentucky Brunch Brand Stout,4.75,['she’s got heat....but man is she somethin ni...
6,Kentucky Brunch Brand Stout,5.00,['Amazing brew. The maple aroma pours out of t...
7,Kentucky Brunch Brand Stout,4.30,"[""Finally got to try the white whale. Pours li..."
8,Kentucky Brunch Brand Stout,4.85,['Had this on tap at the 12-15-18 release in D...
9,Kentucky Brunch Brand Stout,4.89,['Had this on tap At the KBBS release 12/15/18...


In [3]:
import numpy as np
import pandas as pd
import re
import string

In [14]:
# Word frequency calculation for attributes selection
attributes_list = ['aggressive', 'balanced', 'complex', 'crisp', 'fruity', 'hoppy', 'malty', 'robust', 'roasty']
word_freq = dict.fromkeys(attributes_list, 0)

In [15]:
# Generating word frequencies of attributes
for row in beerdf['clean_review']:
  review = row.translate(str.maketrans('', '', string.punctuation)).lower()
  for attribute in attributes_list:
    if attribute in review:
      word_freq[attribute] += 1

In [16]:
word_freq_df = pd.DataFrame.from_dict(word_freq, orient='index', columns=['freq'])
word_freq_df.sort_values('freq', ascending=False)

,freq
complex,768
balanced,748
fruity,443
crisp,299
hoppy,226
roasty,226
malty,167
robust,116
aggressive,89


In [17]:
# Selected attribute set
selected_attribute = ['balanced', 'fruity', 'complex']

# Task C


In [12]:
!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.1.0/en_core_web_lg-2.1.0.tar.gz --no-deps

  Found existing installation: importlib-metadata 0.17
    Uninstalling importlib-metadata-0.17:
      Successfully uninstalled importlib-metadata-0.17
  Stored in directory: C:\Users\saqib\AppData\Local\pip\Cache\wheels\b4\d7\70\426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg


In [20]:
# Load similarity analysis model
import spacy
spacy.cli.download("en_core_web_lg")
similarity_engine = spacy.load("en_core_web_lg")

# Compose the selected attributes as a text
attribute_text = similarity_engine(' '.join(selected_attribute))
review_similarity = beerdf.copy()

# Calculate the similarity score between two texts.
review_similarity['similarity_score'] = review_similarity['clean_review'].apply(lambda x: similarity_engine(x).similarity(attribute_text))
# Sort the result by the similarity score and replace the original one
review_similarity.sort_values('similarity_score', ascending=False, inplace=True)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [22]:
# Export the whole dataset to the file.
review_similarity.to_csv("review_similarity.csv")

In [23]:
# reading the similarity matrix into a dataframe and selecting the top 300 similar reviews.
review_sim = pd.read_csv("review_similarity.csv", index_col=0)
rev_sim_df = pd.DataFrame(review_sim)
rev_sim_df = rev_sim_df.iloc[0:300,:]
rev_sim_df.shape

(300, 4)

In [24]:
rev_sim_df.head()

,Product_name,overall_rating,clean_review,similarity_score
2906,Pseudo Sue,4.40,['Murky creamsicle orange in color. Poured wit...,0.722658
732,Duck Duck Gooze,4.50,['On tap at Beavertown Extravaganza. Aroma has...,0.718163
3980,Saison Bernice,4.44,['Tasted from bottle. Classic farmhouse saison...,0.709067
1083,Double Galaxy,4.37,"['Soft gold, foamy head. Aroma has underripe m...",0.708180
4264,Crusher,4.26,"['Tarnished orange in color with a murky, hazy...",0.706084


# Task D

In [26]:
# Sentiment calculation of the reviews
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import statistics as stats
nltk.download('vader_lexicon')

sentiment = []
sia =SIA()
for sentence in rev_sim_df['clean_review']:
    pol_score=sia.polarity_scores(sentence)
    sentiment.append(pol_score['compound'])

#Add a column sentiment score to the dataframe
rev_sim_df['Sentiment score'] = sentiment
rev_sim_df.sort_values('Sentiment score', ascending=False)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\saqib\AppData\Roaming\nltk_data...


,Product_name,overall_rating,clean_review,similarity_score,Sentiment score
3380,Darkness,4.36,"['750 ml bottle into snifter, bottled on 9/25/...",0.648523,0.9976
1507,Oude Geuze Vintage,4.58,"['750ml bottle, 2008 vintage, poured into a Dr...",0.650559,0.9972
4777,Ghost In The Machine,4.28,"['12 ounce bottle into tulip glass, bottled on...",0.674993,0.9966
3271,Barrel-Aged Sump Coffee Stout,4.39,['Pitch black and nearly opaque with dark choc...,0.670449,0.9965
4942,Emerald Grouper,4.23,"['16 ounce can into tulip glass, canned on 9/2...",0.673370,0.9961
3164,Mocha Wednesday,4.20,"['750 ml bottle into snifter, bottled on 1/27/...",0.649975,0.9959
2348,Beyond Good And Evil,4.22,"['375ml bottle, 2018 release, poured into a Hi...",0.656047,0.9959
1216,Double Dry Hopped Congress Street,4.15,"[""16 ounce can into tulip glass, canned on 4/1...",0.673143,0.9958
4552,Art,4.67,"['375ml bottle, Batch 5, dated March 18, 2015,...",0.662511,0.9950
3435,Montmorency Vs Balaton,4.37,['500 ml bottle into signature tulip glass; Bl...,0.667908,0.9950


# Task E

In [27]:
# weighted score is a combined score of similarity score and the sentiment score
rev_sim_df['Weighted Score'] = (rev_sim_df['similarity_score'] + rev_sim_df['Sentiment score'])/2
rev_sim_df.sort_values(by=['Weighted Score'], inplace=True, ascending=False)

In [30]:
avg_df = rev_sim_df[['Product_name','similarity_score','Sentiment score','Weighted Score']]
avg_df = avg_df.groupby(['Product_name']).mean() #grouped by product name to get an average score for each brand of beer
avg_df.sort_values(by=['Weighted Score'], inplace=True, ascending=False)
avg_df = avg_df.reset_index()
avg_df.head(3)

,Product_name,similarity_score,Sentiment score,Weighted Score
0,Double Dry Hopped Congress Street,0.673143,0.9958,0.834472
1,Barrel-Aged Sump Coffee Stout,0.670449,0.9965,0.833474
2,Last Snow,0.671759,0.9887,0.830230


# Task F

In [32]:
beerdf = pd.read_csv("6039 beer reviews of 250 beers.csv", sep="\t")
beerdf_1= beerdf[['Product_name', 'overall_rating']]
beerdf_1[:10]

,Product_name,overall_rating
0,Kentucky Brunch Brand Stout,4.80
1,Kentucky Brunch Brand Stout,4.74
2,Kentucky Brunch Brand Stout,4.68
3,Kentucky Brunch Brand Stout,5.00
4,Kentucky Brunch Brand Stout,4.97
5,Kentucky Brunch Brand Stout,4.75
6,Kentucky Brunch Brand Stout,5.00
7,Kentucky Brunch Brand Stout,4.30
8,Kentucky Brunch Brand Stout,4.85
9,Kentucky Brunch Brand Stout,4.89


In [33]:
beerdf_2 = beerdf_1.groupby(['Product_name']).mean()
beerdf_2.sort_values(by=['overall_rating'], ascending=False)[:10]

,overall_rating
Product_name,
Kentucky Brunch Brand Stout,4.812000
Chemtrailmix,4.811176
Barrel-Aged Abraxas,4.796400
King JJJuliusss,4.789600
It Was All A Dream,4.785000
Very GGGreennn,4.776818
Zenne Y Frontera,4.764000
Barrel Aged Imperial German Chocolate Cupcake Stout,4.755600
SR-71,4.746400
